In [4]:
import os
import sys

sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"
os.environ["NUMBA_DEBUG_CACHE"] = "1"

# https://github.com/numba/numba/issues/4830 caching problem mixed with 
# https://numba.discourse.group/t/typed-list-of-jitted-functions-in-jitclass/413/4
# maybe its possible to just call the func as func[1]


In [5]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from nb_quantfreedom.nb_enums import DynamicOrderSettingsArrays
from nb_quantfreedom.utils import pretty_qf
from requests import get
from talib import RSI
from numba import njit, types, typed
from numba.experimental import jitclass
import numpy as np
from numba.types import int64, float_, ListType, void
from numba import typeof, typed, njit
from numba.typed import List
from numba.experimental import jitclass
from numba.core.types import FunctionType

# np.set_printoptions(formatter={"float_kind": "{:.2f}".format})

# pd.options.display.float_format = "{:.2f}".format

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
call_str_type = types.void(types.unicode_type)
callee_func_type = types.FunctionType(call_str_type)
sig_str = types.int64(types.int64, types.int64, callee_func_type)


# @numba.njit([sig_str], cache=True)
@njit(cache=True)
def f1(x, y, callee):
    callee("adding x=" + str(x) + " and y=" + str(y))
    return x + y


# @numba.njit([sig_str], cache=True)
@njit(cache=True)
def f2(x, y, callee):
    callee("multiplying x=" + str(x) + " and y=" + str(y))
    return x * y


f_list = typed.List.empty_list(sig_str.as_type())
f_list.append(f1)
f_list.append(f2)


@njit(cache=True)
def callee_str(m):
    print(m)
    pass


@njit(cache=True)
def callee_pass(m):
    pass


p_list = typed.List.empty_list(call_str_type.as_type())
p_list.append(callee_str)
p_list.append(callee_pass)


@jitclass(
    [
        ("funcs", types.ListType(sig_str.as_type())),
        ("printers", types.ListType(call_str_type.as_type())),
    ]
)
class Handler:
    def __init__(self, funcs, printers):
        self.funcs = funcs
        self.printers = printers


@njit(cache=True)
def tester(f_list, p_list):
    x = Handler(f_list, p_list)

    for ind in range(10):
        time: types.int64 = 0
        hello: types.int64 = 0
        for dos in range(200):
            for bar in range(100):
                time += x.funcs[0](ind, dos, x.printers[1]) + bar
                hello += x.funcs[1](ind, 4, x.printers[1])
        result = time - hello
    return result




[cache] index loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\758495555.f1-dac5d5d818.py310.nbi'
[cache] data loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\758495555.f1-dac5d5d818.py310.1.nbc'
[cache] index loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\758495555.f2-eb3e754af9.py310.nbi'
[cache] data loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\758495555.f2-eb3e754af9.py310.1.nbc'
[cache] index loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\758495555.callee_str-e507e98b18.py310.nbi'
[cache] data loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\758495555.callee_str-e507e98b18.py310.1.nbc'
[cache] index loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\758495555.callee_pass-325b917fa4.py310.nbi'
[cache] data loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\758495555.callee_pass-325b917fa4.py310.1.nbc'


In [16]:
tester(f_list=f_list, p_list=p_list)

[cache] index loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\758495555.tester-5ff7167429.py310.nbi'
[cache] data loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\758495555.tester-5ff7167429.py310.1.nbc'


2440000